In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/ar_reviews_100k.tsv', sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Conv1D, GRU, Flatten, Dense, concatenate, GlobalMaxPooling1D, TimeDistributed
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import TFBertForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.layers import Embedding

In [ ]:
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
def bert_tokenizer(text):
    tokens = tokenizer.encode_plus(text,
                                   add_special_tokens=True,
                                   max_length=128,
                                   padding='max_length',
                                   truncation=True,
                                   return_attention_mask=True,
                                   return_tensors='np')

    input_ids = tokens['input_ids'].tolist()
    attention_mask = tokens['attention_mask'].tolist()

    return input_ids, attention_mask

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GRU, LSTM, Dense, Dropout, GlobalAveragePooling1D


In [ ]:
# Split the dataset into train, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)

In [ ]:
# Convert labels to one-hot encoding
label_dict = {label: idx for idx, label in enumerate(df['label'].unique())}
num_classes = len(label_dict)
train_labels = tf.keras.utils.to_categorical(train_labels.map(label_dict))
val_labels = tf.keras.utils.to_categorical(val_labels.map(label_dict))
test_labels = tf.keras.utils.to_categorical(test_labels.map(label_dict))

In [ ]:
# Apply the tokenizer function to each text entry in the DataFrame
train_input_ids, train_attention_mask = zip(*train_texts.map(bert_tokenizer))
val_input_ids, val_attention_mask = zip(*val_texts.map(bert_tokenizer))
test_input_ids, test_attention_mask = zip(*test_texts.map(bert_tokenizer))

# Convert lists to arrays
train_input_ids = np.array(train_input_ids)
train_attention_mask = np.array(train_attention_mask)
val_input_ids = np.array(val_input_ids)
val_attention_mask = np.array(val_attention_mask)
test_input_ids = np.array(test_input_ids)
test_attention_mask = np.array(test_attention_mask)

# Print shapes
print("Train shapes:", train_input_ids.shape, train_attention_mask.shape, train_labels.shape)
print("Validation shapes:", val_input_ids.shape, val_attention_mask.shape, val_labels.shape)
print("Test shapes:", test_input_ids.shape, test_attention_mask.shape, test_labels.shape)

Train shapes: (79999, 1, 128) (79999, 1, 128) (79999, 3)
Validation shapes: (10000, 1, 128) (10000, 1, 128) (10000, 3)
Test shapes: (10000, 1, 128) (10000, 1, 128) (10000, 3)


In [ ]:
# Reshape input_ids and attention_mask arrays to remove the extra dimension
train_input_ids = np.squeeze(train_input_ids, axis=1)
train_attention_mask = np.squeeze(train_attention_mask, axis=1)
val_input_ids = np.squeeze(val_input_ids, axis=1)
val_attention_mask = np.squeeze(val_attention_mask, axis=1)
test_input_ids = np.squeeze(test_input_ids, axis=1)
test_attention_mask = np.squeeze(test_attention_mask, axis=1)

# Print shapes
print("Train shapes:", train_input_ids.shape, train_attention_mask.shape, train_labels.shape)
print("Validation shapes:", val_input_ids.shape, val_attention_mask.shape, val_labels.shape)
print("Test shapes:", test_input_ids.shape, test_attention_mask.shape, test_labels.shape)


Train shapes: (79999, 128) (79999, 128) (79999, 3)
Validation shapes: (10000, 128) (10000, 128) (10000, 3)
Test shapes: (10000, 128) (10000, 128) (10000, 3)


In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, max_seq_length, dropout_rate=0.1, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.head_dim = key_dim // num_heads
        assert key_dim % num_heads == 0
        self.max_seq_length = max_seq_length
        self.dropout_rate = dropout_rate

        self.query_dense = tf.keras.layers.Dense(key_dim)
        self.key_dense = tf.keras.layers.Dense(key_dim)
        self.value_dense = tf.keras.layers.Dense(key_dim)

        self.combine_heads = tf.keras.layers.Dense(key_dim)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        self.positional_encoding = self.get_positional_encoding(max_seq_length, key_dim)

    def call(self, inputs, mask=None):
        query, key, value = inputs

        batch_size = tf.shape(query)[0]
        query += self.positional_encoding
        key += self.positional_encoding
        value += self.positional_encoding
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        query = tf.reshape(query, [batch_size, -1, self.num_heads, self.head_dim])
        query = tf.transpose(query, [0, 2, 1, 3])
        key = tf.reshape(key, [batch_size, -1, self.num_heads, self.head_dim])
        key = tf.transpose(key, [0, 2, 1, 3])
        value = tf.reshape(value, [batch_size, -1, self.num_heads, self.head_dim])
        value = tf.transpose(value, [0, 2, 1, 3])
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores /= tf.math.sqrt(tf.cast(self.key_dim, tf.float32))

        if mask is not None:
            mask = tf.expand_dims(mask, axis=1)
            attention_scores += (mask * -1e9)

        attention_weights = tf.nn.softmax(attention_scores, axis=-1)
        attention_output = tf.matmul(attention_weights, value)

        attention_output = tf.transpose(attention_output, [0, 2, 1, 3])
        attention_output = tf.reshape(attention_output, [batch_size, -1, self.key_dim])

        attention_output = self.combine_heads(attention_output)
        attention_output = self.dropout(attention_output)

        return attention_output

    def get_positional_encoding(self, max_seq_length, key_dim):
        position = tf.range(max_seq_length, dtype=tf.float32)
        position = tf.expand_dims(position, axis=1)
        div_term = tf.exp(tf.range(0, key_dim, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / key_dim))
        positional_encoding = tf.sin(position * div_term)
        positional_encoding = tf.concat([positional_encoding, tf.cos(position * div_term)], axis=1)
        positional_encoding = tf.expand_dims(positional_encoding, axis=0)
        return positional_encoding

In [ ]:
from tensorflow.keras.layers import LSTM, GRU, Dense, Input, Concatenate, GlobalAveragePooling1D, Dropout
from tensorflow.keras.models import Model

In [ ]:
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

embedding = Embedding(input_dim=tokenizer.vocab_size, output_dim=128, input_length=128)(input_ids)

attention_output = MultiHeadAttention(num_heads=4, key_dim=128, max_seq_length=128)([embedding, embedding, embedding])

lstm_output = LSTM(64, return_sequences=True)(attention_output)
gru_output = GRU(64, return_sequences=True)(attention_output)

lstm_pooled = GlobalAveragePooling1D()(lstm_output)
gru_pooled = GlobalAveragePooling1D()(gru_output)

concatenated_output = Concatenate()([lstm_pooled, gru_pooled])

# Dense layers for classification
dense = Dense(64, activation='relu')(concatenated_output)
dropout = Dropout(0.2)(dense)
output = Dense(3, activation='softmax')(dropout)

model = Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 128, 128)             3906816   ['input_ids[0][0]']           
                                                                                                  
 multi_head_attention (Mult  (None, None, 128)            66048     ['embedding[0][0]',           
 iHeadAttention)                                                     'embedding[0][0]',           
                                                                     'embedding[0][0]']           
                                                                                              

In [ ]:
epochs = 5
batch_size = 32

# Train the model
history = model.fit(
    x=[train_input_ids, train_attention_mask],  # Input data
    y=train_labels,  # Target labels
    epochs=epochs,  # Number of epochs
    batch_size=batch_size,  # Batch size
    validation_data=([val_input_ids, val_attention_mask], val_labels)  # Validation data
)

Epoch 1/5
 395/2500 [===>..........................] - ETA: 13:25 - loss: 1.0997 - accuracy: 0.3362

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, Dropout, LayerNormalization, MultiHeadAttention


In [ ]:
# Define input layers for input_ids and attention_mask
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

In [ ]:
# Define the Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
# Define the Transformer model
def create_transformer_model(input_shape, vocab_size, embed_dim=128, num_heads=4, ff_dim=128, num_transformer_blocks=2, rate=0.1):
    inputs = [input_ids, attention_mask]  # Group input layers into a list
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs[0])  # Use input_ids
    transformer_blocks = [TransformerBlock(embed_dim, num_heads, ff_dim, rate) for _ in range(num_transformer_blocks)]
    x = embedding_layer
    for transformer_block in transformer_blocks:
        x = transformer_block(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define input shape and vocabulary size
input_shape = (128,)
vocab_size = tokenizer.vocab_size  # Assuming you have a tokenizer object

# Create the Transformer model
transformer_model = create_transformer_model(input_shape, vocab_size)

# Compile the model
transformer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
transformer_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_5 (Embedding)     (None, 128, 128)             3906816   ['input_ids[0][0]']           
                                                                                                  
 transformer_block_6 (Trans  (None, 128, 128)             297344    ['embedding_5[0][0]']         
 formerBlock)                                                                                     
                                                                                                  
 transformer_block_7 (Trans  (None, 128, 128)             297344    ['transformer_block_6[0]

In [ ]:
history = transformer_model.fit(
    x=[train_input_ids, train_attention_mask],
    y=train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([val_input_ids, val_attention_mask], val_labels)
)

Epoch 1/5
2500/2500 [==============================] - 67s 23ms/step - loss: 1.0461 - accuracy: 0.4435 - val_loss: 1.0030 - val_accuracy: 0.4744
Epoch 2/5
2500/2500 [==============================] - 51s 20ms/step - loss: 1.0086 - accuracy: 0.4794 - val_loss: 1.0005 - val_accuracy: 0.4887
Epoch 3/5
2500/2500 [==============================] - 51s 21ms/step - loss: 1.0026 - accuracy: 0.4845 - val_loss: 1.0017 - val_accuracy: 0.4874
Epoch 4/5
2500/2500 [==============================] - 51s 20ms/step - loss: 0.9991 - accuracy: 0.4875 - val_loss: 0.9950 - val_accuracy: 0.4990
Epoch 5/5
2500/2500 [==============================] - 50s 20ms/step - loss: 0.9967 - accuracy: 0.4883 - val_loss: 0.9890 - val_accuracy: 0.4934


In [ ]:
# Define input layers for input_ids and attention_mask
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Define the Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
# Define the Transformer model
def create_transformer_gru_model(input_shape, vocab_size, embed_dim=128, num_heads=4, ff_dim=128, num_transformer_blocks=2, rate=0.1):
    inputs = [input_ids, attention_mask]  # Group input layers into a list
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs[0])  # Use input_ids
    transformer_blocks = [TransformerBlock(embed_dim, num_heads, ff_dim, rate) for _ in range(num_transformer_blocks)]
    x = embedding_layer
    for transformer_block in transformer_blocks:
        x = transformer_block(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    gru_output = GRU(64, return_sequences=True)(embedding_layer)  # Add GRU layer
    x = GlobalAveragePooling1D()(gru_output)
    outputs = Dense(3, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define input shape and vocabulary size
input_shape = (128,)
vocab_size = tokenizer.vocab_size  # Assuming you have a tokenizer object

# Create the Transformer model with GRU
transformer_gru_model = create_transformer_gru_model(input_shape, vocab_size)

# Compile the model
transformer_gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
transformer_gru_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, 128, 128)             3906816   ['input_ids[0][0]']           
                                                                                                  
 gru_1 (GRU)                 (None, 128, 64)              37248     ['embedding_6[0][0]']         
                                                                                                  
 global_average_pooling1d_6  (None, 64)                   0         ['gru_1[0][0]']               
  (GlobalAveragePooling1D)                                                                  

In [ ]:
history = transformer_gru_model.fit(
    x=[train_input_ids, train_attention_mask],
    y=train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([val_input_ids, val_attention_mask], val_labels)
)

Epoch 1/5
2500/2500 [==============================] - 573s 228ms/step - loss: 0.9049 - accuracy: 0.5293 - val_loss: 0.8238 - val_accuracy: 0.5753
Epoch 2/5
2500/2500 [==============================] - 545s 218ms/step - loss: 0.8184 - accuracy: 0.5849 - val_loss: 0.7996 - val_accuracy: 0.5930
Epoch 3/5
2500/2500 [==============================] - 532s 213ms/step - loss: 0.7945 - accuracy: 0.6037 - val_loss: 0.7793 - val_accuracy: 0.6074
Epoch 4/5
2500/2500 [==============================] - 552s 221ms/step - loss: 0.7726 - accuracy: 0.6169 - val_loss: 0.7688 - val_accuracy: 0.6135
Epoch 5/5
2500/2500 [==============================] - 557s 223ms/step - loss: 0.7549 - accuracy: 0.6295 - val_loss: 0.7519 - val_accuracy: 0.6310


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, Dropout, GRU, LSTM, Conv1D, MaxPooling1D, Concatenate


In [ ]:
# Define input layers for input_ids and attention_mask
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Define the Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
# Define the model
def create_transformer_gru_lstm_cnn_model(input_shape, vocab_size, embed_dim=128, num_heads=4, ff_dim=128, num_transformer_blocks=2, gru_units=64, lstm_units=64, cnn_filters=64, cnn_kernel_size=3, rate=0.1):
    inputs = [input_ids, attention_mask]  # Group input layers into a list
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs[0])  # Use input_ids
    transformer_blocks = [TransformerBlock(embed_dim, num_heads, ff_dim, rate) for _ in range(num_transformer_blocks)]
    x = embedding_layer
    for transformer_block in transformer_blocks:
        x = transformer_block(x)
    x = GlobalAveragePooling1D()(x)

    # CNN Layer
    cnn_output = Conv1D(cnn_filters, cnn_kernel_size, activation='relu')(embedding_layer)
    cnn_output = MaxPooling1D()(cnn_output)
    cnn_output = GlobalAveragePooling1D()(cnn_output)

    # Dense layers for classification
    x = Dense(64, activation='relu')(cnn_output)
    outputs = Dense(3, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define input shape and vocabulary size
input_shape = (128,)
vocab_size = tokenizer.vocab_size  # Assuming you have a tokenizer object

# Create the model
model = create_transformer_gru_lstm_cnn_model(input_shape, vocab_size)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_7 (Embedding)     (None, 128, 128)             3906816   ['input_ids[0][0]']           
                                                                                                  
 conv1d (Conv1D)             (None, 126, 64)              24640     ['embedding_7[0][0]']         
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 63, 64)               0         ['conv1d[0][0]']              
 D)                                                                                         

In [ ]:
history = model.fit(
    x=[train_input_ids, train_attention_mask],
    y=train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([val_input_ids, val_attention_mask], val_labels)
)

Epoch 1/5
2500/2500 [==============================] - 20s 7ms/step - loss: 0.9435 - accuracy: 0.5238 - val_loss: 0.8529 - val_accuracy: 0.5856
Epoch 2/5
2500/2500 [==============================] - 13s 5ms/step - loss: 0.8275 - accuracy: 0.5982 - val_loss: 0.8142 - val_accuracy: 0.6052
Epoch 3/5
2500/2500 [==============================] - 13s 5ms/step - loss: 0.7960 - accuracy: 0.6176 - val_loss: 0.8134 - val_accuracy: 0.5985
Epoch 4/5
2500/2500 [==============================] - 14s 6ms/step - loss: 0.7779 - accuracy: 0.6290 - val_loss: 0.7851 - val_accuracy: 0.6212
Epoch 5/5
2500/2500 [==============================] - 14s 6ms/step - loss: 0.7666 - accuracy: 0.6324 - val_loss: 0.7951 - val_accuracy: 0.6265


In [ ]:
# Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

history = model.fit(
    x=[train_input_ids, train_attention_mask],
    y=train_labels,
    epochs=10,
    batch_size=16,
    validation_data=([val_input_ids, val_attention_mask], val_labels)
)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_7 (Embedding)     (None, 128, 128)             3906816   ['input_ids[0][0]']           
                                                                                                  
 conv1d (Conv1D)             (None, 126, 64)              24640     ['embedding_7[0][0]']         
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 63, 64)               0         ['conv1d[0][0]']              
 D)                                                                                         

In [ ]:
# Extract model weights
model_weights = model.get_weights()

In [ ]:
# Define the BERT model architecture
def create_bert_model(max_seq_length, vocab_size):
    # Input layer
    input_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_ids")

    # Embedding layer
    embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128)(input_ids)

    # Transformer layers (you can adjust the number of layers)
    num_layers = 4
    num_heads = 4
    feed_forward_dim = 512
    dropout_rate = 0.1

    for _ in range(num_layers):
        # Multi-head self-attention
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=128)(embedding_layer, embedding_layer)
        attention_output = tf.keras.layers.Dropout(rate=dropout_rate)(attention_output)
        attention_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(embedding_layer + attention_output)

        # Feed-forward neural network
        ffn_output = tf.keras.layers.Dense(units=feed_forward_dim, activation='relu')(attention_output)
        ffn_output = tf.keras.layers.Dense(units=128)(ffn_output)
        ffn_output = tf.keras.layers.Dropout(rate=dropout_rate)(ffn_output)
        embedding_layer = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)

    # Classification layers
    pooled_output = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
    pooled_output = tf.keras.layers.Dense(units=128, activation='relu')(pooled_output)
    pooled_output = tf.keras.layers.Dropout(rate=dropout_rate)(pooled_output)
    outputs = tf.keras.layers.Dense(units=num_classes, activation='softmax')(pooled_output)

    # Define the model
    model = Model(inputs=input_ids, outputs=outputs)

    return model

# Example usage
max_seq_length = 128
vocab_size = 10000
num_classes = 3

bert_model = create_bert_model(max_seq_length, vocab_size)
bert_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_9 (Embedding)     (None, 128, 128)             1280000   ['input_ids[0][0]']           
                                                                                                  
 multi_head_attention_16 (M  (None, 128, 128)             263808    ['embedding_9[0][0]',         
 ultiHeadAttention)                                                  'embedding_9[0][0]']         
                                                                                                  
 dropout_31 (Dropout)        (None, 128, 128)             0         ['multi_head_attention_1

In [ ]:
bert_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = bert_model.fit(
    x=train_input_ids,
    y=train_labels,
    epochs=10,
    batch_size=16,
    validation_data=(val_input_ids, val_labels)
)

Epoch 1/10
5000/5000 [==============================] - 185s 35ms/step - loss: 1.1011 - accuracy: 0.3342 - val_loss: 1.0987 - val_accuracy: 0.3354
Epoch 2/10
5000/5000 [==============================] - 127s 25ms/step - loss: 1.0987 - accuracy: 0.3333 - val_loss: 1.0988 - val_accuracy: 0.3354
Epoch 3/10
5000/5000 [==============================] - 127s 25ms/step - loss: 1.0987 - accuracy: 0.3327 - val_loss: 1.0989 - val_accuracy: 0.3292
Epoch 4/10
5000/5000 [==============================] - 125s 25ms/step - loss: 1.0987 - accuracy: 0.3318 - val_loss: 1.0987 - val_accuracy: 0.3354
Epoch 5/10
5000/5000 [==============================] - 125s 25ms/step - loss: 1.0987 - accuracy: 0.3314 - val_loss: 1.0986 - val_accuracy: 0.3292
Epoch 6/10
5000/5000 [==============================] - 126s 25ms/step - loss: 1.0987 - accuracy: 0.3301 - val_loss: 1.0986 - val_accuracy: 0.3354
Epoch 7/10
4999/5000 [============================>.] - ETA: 0s - loss: 1.0987 - accuracy: 0.3333

KeyboardInterrupt: 

In [ ]:
# create attention embedder with q, k, v
class AttentionEmbedder(tf.keras.Model):
    def __init__(self, units):
        super(AttentionEmbedder, self).__init__()
        self.Wq = tf.keras.layers.Dense(units)
        self.Wk = tf.keras.layers.Dense(units)
        self.Wv = tf.keras.layers.Dense(units)
        self.units = units

    def call(self, inputs):
        q = self.Wq(inputs)
        k = self.Wk(inputs)
        v = self.Wv(inputs)
        return q, k, v

In [ ]:
# create attention layer
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def call(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [ ]:
# Define input layers
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Define vocabulary size and embedding dimension
vocab_size = tokenizer.vocab_size
embed_dim = 128  # Example embedding dimension

# Create embedding layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(input_ids)

# Apply AttentionEmbedder layer
attention_embedder = AttentionEmbedder(units=64)
q, k, v = attention_embedder(embedding_layer)

# Apply Attention layer
attention = Attention()
attention_output, attention_weights = attention(q, k, v, mask=attention_mask)

# Global average pooling
pooled_output = GlobalAveragePooling1D()(attention_output)

# Dense layers for classification
dense = Dense(64, activation='relu')(pooled_output)
output = Dense(3, activation='softmax')(dense)

# Define and compile the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 embedding_17 (Embedding)    (None, 128, 128)             4096000   ['input_ids[0][0]']           
                                                                                                  
 attention_embedder_7 (Atte  ((None, 128, 64),            24768     ['embedding_17[0][0]']        
 ntionEmbedder)               (None, 128, 64),                                                    
                              (None, 128, 64))                                                    
                                                                                            

In [ ]:
# Train the model
history = model.fit(
    x=[train_input_ids, train_attention_mask],
    y=train_labels,
    epochs=10,
    batch_size=16,
    validation_data=([val_input_ids, val_attention_mask], val_labels)
)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node model_8/attention_7/add defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pytho

In [ ]:
print(train_input_ids.shape)
print(train_attention_mask.shape)

(79999, 128)
(79999, 128)


In [ ]:
print(val_input_ids.shape)
print(val_attention_mask.shape)

(10000, 128)
(10000, 128)


In [ ]:
for layer in model.layers:
    print(layer.input_shape)